In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
%pip install databricks-sdk --upgrade

In [0]:
%restart_python

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import EndpointCoreConfigInput, ServedEntityInput, ServingModelWorkloadType, AiGatewayConfig, AiGatewayInferenceTableConfig, AiGatewayUsageTrackingConfig

w = WorkspaceClient()

In [0]:
w.serving_endpoints.create(
    name = model_name
    ,config = EndpointCoreConfigInput(
        name = model_name
        ,served_entities = [
            ServedEntityInput(
                entity_name = full_model_name
                ,entity_version = client.get_model_version_by_alias(full_model_name, "latest_version").version
                ,environment_vars = {
                    'REDOX_PRIVATE_KEY': "{{secrets/" + f"{databricks_secret_scope}" + "/redox_private_key}}"
                    ,'REDOX_CLIENT_ID': "{{secrets/" + f"{databricks_secret_scope}" + "/redox_client_id}}"
                    ,'REDOX_SOURCE_ID': "{{secrets/" + f"{databricks_secret_scope}" + "/redox_source_id}}"
                    ,'REDOX_PUBLIC_KID': "{{secrets/" + f"{databricks_secret_scope}" + "/redox_public_kid}}"
                }
                ,scale_to_zero_enabled = True
                ,workload_size = "Small"
                ,workload_type = ServingModelWorkloadType("CPU")
            )
        ]
    )
    ,ai_gateway = AiGatewayConfig(
        inference_table_config = AiGatewayInferenceTableConfig(
            catalog_name=catalog_use
            ,schema_name=schema_use
            ,table_name_prefix = None
            ,enabled=True
        )
        ,usage_tracking_config = AiGatewayUsageTrackingConfig(
            enabled=True
        )
    )
)